In [ ]:
import pyimzml.ImzMLParser
import tifffile
import matplotlib.pyplot as plt
import numpy as np
import cv2

def interpolate_MSI(filename,image_path,msi_image,smoothed_image,output_directory,sample_name):
    parser = pyimzml.ImzMLParser.ImzMLParser(filename)
    # Get dimensions
    x_dimension = parser.imzmldict['max count of pixels x']
    y_dimension = parser.imzmldict['max count of pixels y']
    #print('MSI Dimensions:\n')
    #print(f"Dimensions: {x_dimension} x {y_dimension}")

    x_resolution = None
    y_resolution = None

    tif_file = image_path
    # Read the TIFF file and get metadata
    with tifffile.TiffFile(tif_file) as tif:
        # Get dimensions
        x_dimension = tif.pages[0].shape[1]
        y_dimension = tif.pages[0].shape[0]
        #print('\nH&E dimensions:\n')
        #print(f"Dimensions: {x_dimension} x {y_dimension}")

    binarized_image = smoothed_image

    # Ensure the image is binary (0 or 255)
    binarized_image[binarized_image > 0] = 1 

    # Calculate row and column sums
    row_sum = np.sum(binarized_image, axis=1)  # Sum along rows
    col_sum = np.sum(binarized_image, axis=0)  # Sum along columns

    crop_threshold = 15

    # Determine crop boundaries
    top_crop = np.argmax(row_sum > crop_threshold)          # Top boundary
    bottom_crop = len(row_sum) - np.argmax(row_sum[::-1] > crop_threshold)  # Bottom boundary
    left_crop = np.argmax(col_sum > crop_threshold)         # Left boundary
    right_crop = len(col_sum) - np.argmax(col_sum[::-1] > crop_threshold)  # Right boundary

    # Crop the image
    cropped_image = binarized_image[top_crop:bottom_crop, left_crop:right_crop]

    #fig = px.imshow(cropped_image,title='Cropped H&E')
    #fig.show()

    #fig_outpath = output_directory + '\\' + sample_name + '_HE_cropped_image.png'
    #fig.write_image(fig_outpath)

    dimensions = cropped_image.shape

    #print("Dimensions:", dimensions)
    #print("Number of rows:", dimensions[0])
    #print("Number of columns:", dimensions[1]) 
    
    dimensions = cropped_image.shape

    x_dimension = dimensions[1]
    y_dimension = dimensions[0]

    #print(x_dimension,y_dimension)

    # Resize MSI image to match dimensions of microscopy image
    resized_msi_image = cv2.resize(msi_image, (x_dimension, y_dimension), interpolation=cv2.INTER_LINEAR)

    #resized_msi_image = cv2.resize(msi_image, (928, 1990), interpolation=cv2.INTER_LINEAR)

    plt.figure(figsize=(10, 8))
    plt.imshow(resized_msi_image)
    plt.colorbar()
    title = 'Resized MSI image with linear interpolation'
    plt.title(title)

    fig_outpath = output_directory + '\\' + sample_name + '_MSI_composite_image_all_mz.png'
    plt.savefig(fig_outpath,bbox_inches='tight')
    
    return cropped_image,resized_msi_image